In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 14.7 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
!nvidia-smi  

Tue Jun 28 22:46:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_481.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_480_trial_1.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118689,0.037046,0.048969,0.056781,0.056370,0.061304,0.062537,0.055959,0.034579,0.008676,-0.020928,...,-0.032029,-0.019283,-0.017227,-0.022161,-0.015172,-0.027095,-0.018461,-0.018050,-0.036963,2
118690,-0.015745,-0.021556,-0.015261,-0.030756,-0.022040,-0.024461,-0.047703,-0.056418,-0.041892,-0.023493,...,-0.057871,-0.026882,-0.010419,0.020086,0.059790,0.052527,0.047201,0.030738,-0.015261,2
118691,0.006209,0.014432,0.021010,0.040746,0.014021,0.009909,0.021833,0.001275,-0.018050,-0.014349,...,0.007031,0.028822,0.031289,0.004564,0.018955,0.030056,0.007031,-0.028740,-0.013527,0
118692,-0.019135,-0.091765,-0.111133,-0.129533,-0.170206,-0.202163,-0.230247,-0.283025,-0.287382,-0.299003,...,-0.039471,-0.042376,-0.035598,0.004591,-0.021072,-0.048671,-0.048671,-0.004125,-0.004125,2


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 13.5 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_dt.shape

(118694, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118689,0.037046,0.048969,0.056781,0.056370,0.061304,0.062537,0.055959,0.034579,0.008676,-0.020928,...,-0.032440,-0.032029,-0.019283,-0.017227,-0.022161,-0.015172,-0.027095,-0.018461,-0.018050,-0.036963
118690,-0.015745,-0.021556,-0.015261,-0.030756,-0.022040,-0.024461,-0.047703,-0.056418,-0.041892,-0.023493,...,-0.050608,-0.057871,-0.026882,-0.010419,0.020086,0.059790,0.052527,0.047201,0.030738,-0.015261
118691,0.006209,0.014432,0.021010,0.040746,0.014021,0.009909,0.021833,0.001275,-0.018050,-0.014349,...,-0.007360,0.007031,0.028822,0.031289,0.004564,0.018955,0.030056,0.007031,-0.028740,-0.013527
118692,-0.019135,-0.091765,-0.111133,-0.129533,-0.170206,-0.202163,-0.230247,-0.283025,-0.287382,-0.299003,...,-0.021556,-0.039471,-0.042376,-0.035598,0.004591,-0.021072,-0.048671,-0.048671,-0.004125,-0.004125


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(118694,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.00620862],
        [ 0.0144318 ],
        [ 0.02101034],
        ...,
        [ 0.00703094],
        [-0.02873989],
        [-0.01352701]],

       [[-0.01913479],
        [-0.09176502],
        [-0.11113308],
        ...,
        [-0.04867109],
        [-0.00412455],
        [-0.00412455]],

       [[-0.01429278],
        [ 0.01379091],
        [ 0.0118541 ],
        ...,
        [-0.02785042],
        [-0.05012369],
        [-0.0210716 ]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-xsfh1hxi
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-xsfh1hxi
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=1d8613f89a4de5add8b96d7586bae7337df00b903a7f9e536baaf8ad303c2192
  Stored in directory: /tmp/pip-ephem-wheel-cache-72iebt04/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt
optimizer=Adam(lr=1e-4)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      model=model_b(verbose=VBS)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      optimizer = Adam(lr=1e-4)
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_dt_rshp, y_dt_,
            epochs=hp.Choice("epochs", [40, 50, 60, 70])
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tuner.search(x_dt_rshp, y_dt_)

Trial 3 Complete [00h 57m 15s]
loss: 0.07384231314063072

Best loss So Far: 0.057738400995731354
Total elapsed time: 03h 08m 00s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 70
Score: 0.057738400995731354
Trial summary
Hyperparameters:
epochs: 50
Score: 0.07384231314063072
Trial summary
Hyperparameters:
epochs: 40
Score: 0.08827489241957664


Trial 1(Teacher): 

tuner.search() o/p

Trial 3 Complete [00h 58m 27s]

loss: 0.07305121794342995

Best loss So Far: 0.07305121794342995

Total elapsed time: 02h 13m 01s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() o/p:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f16e0b19810>

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07305121794342995

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08693895861506462

Trial summary

Hyperparameters:

epochs: 40

Score: 0.087614044547081

Trial 2(Teacher): 

tuner.search() o/p

Trial 3 Complete [00h 58m 27s]

loss: 0.06456424668431282

Best loss So Far: 0.06456424668431282

Total elapsed time: 02h 23m 24s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() o/p:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f16e0b19810>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06456424668431282

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07464605197310448

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08797243982553482

Trial 3(Teacher):

tuner.search() O/P:

Trial 3 Complete [00h 57m 15s]

loss: 0.07384231314063072

Best loss So Far: 0.057738400995731354

Total elapsed time: 03h 08m 00s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() O/P:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f36ccbbc350>

Trial summary

Hyperparameters:

epochs: 70

Score: 0.057738400995731354 (not taking this one, coz there is too large a diff between this loss and the 50 epochs loss (could be overfitting)), hence going with 50 epochs

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07384231314063072

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08827489241957664

In [ ]:
#Best model : 60 epochs teacher

In [ ]:
teacher_40_epochs=model_b(verbose=VBS)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_40_epochs.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Training the teacher model for 40 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_40_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_40_epochs_dir = os.path.dirname(checkpoint_teacher_model_40_epochs)

In [ ]:
cp_callback_teacher_model_40_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
teacher_40_epochs.fit(x_dt_rshp, y_dt_, epochs=40, callbacks=[cp_callback_teacher_model_40_epochs])

Epoch 1/40
3710/3710 [==============================] - 39s 9ms/step - loss: 0.6457 - accuracy: 0.7458
Epoch 2/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.3778 - accuracy: 0.8512
Epoch 3/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.3192 - accuracy: 0.8739
Epoch 4/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.2789 - accuracy: 0.8902
Epoch 5/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.2493 - accuracy: 0.9024
Epoch 6/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.2273 - accuracy: 0.9102
Epoch 7/40
3710/3710 [==============================] - 35s 9ms/step - loss: 0.2091 - accuracy: 0.9168
Epoch 8/40
3710/3710 [==============================] - 33s 9ms/step - loss: 0.1948 - accuracy: 0.9226
Epoch 9/40
3710/3710 [==============================] - 33s 9ms/step - loss: 0.1863 - accuracy: 0.9263
Epoch 10/40
3710/3710 [==============================] - 32s 9ms/step - l

In [ ]:
latest_teacher_40_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_40_epochs_dir)

In [ ]:
print(latest_teacher_40_epochs)
print(checkpoint_teacher_model_40_epochs)

/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt


In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(494, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.052176,-0.045597,-0.036963,-0.035730,-0.047242,-0.048476,-0.041486,-0.040252,-0.050942,-0.046420,...,-0.003659,0.024711,0.084740,0.123389,0.139013,0.135313,0.129556,0.137779,0.111054,0.078573
1,0.080611,0.077706,0.053495,0.029285,0.016212,-0.007998,-0.003156,-0.036082,-0.003156,0.032191,...,-0.015745,0.037033,0.049622,0.004107,0.013307,0.012823,0.022991,0.083032,0.043811,0.011854
2,-0.113070,-0.092733,-0.155195,-0.101449,-0.073365,-0.013809,-0.100965,-0.026398,-0.035113,0.017665,...,0.007496,0.025896,-0.104354,-0.073365,-0.081597,-0.063681,0.022507,-0.005577,-0.059808,-0.090312
3,0.036064,0.019601,0.002170,0.012823,0.028317,0.019601,0.017665,0.017665,0.025412,0.006528,...,-0.033177,-0.080144,-0.080628,-0.053029,-0.031724,-0.018651,0.003623,-0.007514,-0.007514,-0.019619
4,0.042802,0.047736,0.055137,0.051436,0.048558,0.041157,0.053081,0.053081,0.059659,0.058837,...,-0.021750,-0.033263,-0.056288,-0.055876,-0.063688,-0.067389,-0.067800,-0.052998,-0.038197,-0.029562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,-0.024461,-0.008482,-0.022040,-0.016714,-0.002188,0.011854,-0.008967,0.008465,0.034127,-0.007030,...,-0.052060,-0.057387,-0.014293,0.018633,-0.133406,-0.057871,-0.066102,-0.011388,-0.033661,-0.044313
490,0.058337,0.068506,0.054948,0.058822,0.104337,0.088358,0.056401,0.054464,0.050590,0.069474,...,-0.021556,-0.004609,-0.038987,-0.034145,-0.008967,0.037517,-0.017198,0.033643,0.074316,0.048169
491,0.016899,0.047325,0.062537,0.066238,0.058015,0.027589,-0.015583,-0.037374,-0.045597,-0.024217,...,-0.004482,-0.031207,-0.028740,-0.012705,-0.007771,-0.019694,-0.020106,-0.007771,-0.019694,-0.025451
492,0.310877,0.331846,0.351582,0.317045,0.309233,0.333491,0.301010,0.272228,0.282096,0.287030,...,0.065416,0.065827,0.062126,0.055137,0.032523,0.037457,0.045680,0.061715,0.028000,0.020188


In [ ]:
x_test_dt.shape

(494, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(494,3000,1)

In [ ]:
x_test_dt_rshp

array([[[-0.05217595],
        [-0.04559741],
        [-0.03696307],
        ...,
        [ 0.13777948],
        [ 0.11105415],
        [ 0.07857259]],

       [[ 0.08061072],
        [ 0.07770551],
        [ 0.05349543],
        ...,
        [ 0.08303172],
        [ 0.0438114 ],
        [ 0.0118541 ]],

       [[-0.11306988],
        [-0.09273342],
        [-0.15519542],
        ...,
        [-0.00557715],
        [-0.05980772],
        [-0.09031241]],

       ...,

       [[ 0.01689875],
        [ 0.04732451],
        [ 0.06253739],
        ...,
        [-0.00777079],
        [-0.01969439],
        [-0.02545062]],

       [[ 0.31087739],
        [ 0.33184649],
        [ 0.35158212],
        ...,
        [ 0.06171507],
        [ 0.02800004],
        [ 0.02018802]],

       [[-0.02494521],
        [-0.00799816],
        [-0.03705025],
        ...,
        [ 0.05010602],
        [ 0.0123383 ],
        [ 0.02299074]]])

In [ ]:
accuracy_teacher_model_40_epochs = teacher_40_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

Training the teacher model for 50 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_50_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_50_epochs_dir = os.path.dirname(checkpoint_teacher_model_50_epochs)

In [ ]:
cp_callback_teacher_model_50_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_50_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=50)

In [ ]:
teacher_50_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_50_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_50_epochs.fit(x_dt_rshp, y_dt_, epochs=50, callbacks=[cp_callback_teacher_model_50_epochs])

Epoch 1/50
3710/3710 [==============================] - 45s 8ms/step - loss: 0.6735 - accuracy: 0.7279
Epoch 2/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.3984 - accuracy: 0.8429
Epoch 3/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.3292 - accuracy: 0.8698
Epoch 4/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2855 - accuracy: 0.8873
Epoch 5/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2528 - accuracy: 0.9008
Epoch 6/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2304 - accuracy: 0.9086
Epoch 7/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2127 - accuracy: 0.9166
Epoch 8/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1971 - accuracy: 0.9217
Epoch 9/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1884 - accuracy: 0.9254
Epoch 10/50
3710/3710 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_50_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_50_epochs_dir)

In [ ]:
accuracy_teacher_model_50_epochs = teacher_50_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 8ms/step - loss: 1.7669 - accuracy: 0.7166


Training the teacher model for 60 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_60_epochs_dir = os.path.dirname(checkpoint_teacher_model_60_epochs)

In [ ]:
cp_callback_teacher_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
teacher_60_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_60_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_60_epochs.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])

Epoch 1/60
3710/3710 [==============================] - 46s 8ms/step - loss: 0.6720 - accuracy: 0.7300
Epoch 2/60
3710/3710 [==============================] - 31s 8ms/step - loss: 0.3961 - accuracy: 0.8441
Epoch 3/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.3273 - accuracy: 0.8702
Epoch 4/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.2835 - accuracy: 0.8878
Epoch 5/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.2507 - accuracy: 0.9013
Epoch 6/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.2278 - accuracy: 0.9098
Epoch 7/60
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2114 - accuracy: 0.9168
Epoch 8/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.1963 - accuracy: 0.9223
Epoch 9/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.1882 - accuracy: 0.9251
Epoch 10/60
3710/3710 [==============================] - 30s 8ms/step - l

In [ ]:
latest_teacher_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_60_epochs_dir)

In [ ]:
accuracy_teacher_model_60_epochs = teacher_60_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 9ms/step - loss: 1.7056 - accuracy: 0.7267


training the teacher model for 70 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_70_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_70_epochs_dir = os.path.dirname(checkpoint_teacher_model_70_epochs)

In [ ]:
cp_callback_teacher_model_70_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_70_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=70)

In [ ]:
teacher_70_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_70_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_70_epochs.fit(x_dt_rshp, y_dt_, epochs=70, callbacks=[cp_callback_teacher_model_70_epochs])

Epoch 1/70
3710/3710 [==============================] - 46s 8ms/step - loss: 0.6721 - accuracy: 0.7297
Epoch 2/70
3710/3710 [==============================] - 29s 8ms/step - loss: 0.3964 - accuracy: 0.8443
Epoch 3/70
3710/3710 [==============================] - 28s 8ms/step - loss: 0.3269 - accuracy: 0.8704
Epoch 4/70
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2868 - accuracy: 0.8865
Epoch 5/70
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2535 - accuracy: 0.9005
Epoch 6/70
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2306 - accuracy: 0.9088
Epoch 7/70
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2125 - accuracy: 0.9161
Epoch 8/70
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1984 - accuracy: 0.9214
Epoch 9/70
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1901 - accuracy: 0.9248
Epoch 10/70
3710/3710 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_70_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_70_epochs_dir)

In [ ]:
accuracy_teacher_model_70_epochs = teacher_70_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 9ms/step - loss: 1.7825 - accuracy: 0.7227


Trial 2:

Loss diff: 40 epochs - 50 epochs: 0.0879-0.0736--> 0.0143

Loss diff: 50 eochs - 60 epochs: 0.0736 - 0.0647 ---> 0.0089

Loss diff: 60 epochs: 70 epochs: 0.0647 - 0.0588 --> 0.0059

Trial 1

KD using the 50 epochs teacher model, finalized the teacher with 60 epochs student model: alpha: 0.9, temp: 5

Trial 2

KD using the 60 epochs teacher model, finalized the teacher with 70 epochs student model: alpha: 0.9, temp: 5

Trial 3

KD using the  epochs teacher model, finalized the teacher with  epochs, student model: alpha: 0.9, temp: 5

KD with 40 epochs

In [ ]:
teacher_model=model_b(verbose=VBS) #40 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 1.5973 - accuracy: 0.7287


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.64285714, 0.11392405, 0.94713656, 0.4       , 0.92982456])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.6020413318771114
Recall: 0.606748463754043
F1 score: 0.5849502897257989
              precision    recall  f1-score   support

           0   0.941860  0.642857  0.764151       126
           1   0.195652  0.113924  0.144000        79
           2   0.820611  0.947137  0.879346       227
           3   0.500000  0.400000  0.444444         5
           4   0.552083  0.929825  0.692810        57

    accuracy                       0.728745       494
   macro avg   0.602041  0.606748  0.584950       494
weighted avg   0.717365  0.728745  0.706443       494



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=50)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/50
3710/3710 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6979 - student_loss: 0.7379 - distillation_loss: 0.0015
Epoch 2/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7644 - student_loss: 0.5633 - distillation_loss: 0.0012
Epoch 3/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7783 - student_loss: 0.5319 - distillation_loss: 0.0011
Epoch 4/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7870 - student_loss: 0.5144 - distillation_loss: 0.0011
Epoch 5/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7940 - student_loss: 0.4957 - distillation_loss: 0.0010
Epoch 6/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8000 - student_loss: 0.4829 - distillation_loss: 0.0010
Epoch 7/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.7024 - student_loss: 1.1366


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.74603175, 0.10126582, 0.84140969, 0.4       , 0.9122807 ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.570952293622167
Recall: 0.6001975924401796
F1 score: 0.5522034718412046
              precision    recall  f1-score   support

           0   0.930693  0.746032  0.828194       126
           1   0.285714  0.101266  0.149533        79
           2   0.864253  0.841410  0.852679       227
           3   0.400000  0.400000  0.400000         5
           4   0.374101  0.912281  0.530612        57

    accuracy                       0.702429       494
   macro avg   0.570952  0.600198  0.552203       494
weighted avg   0.727425  0.702429  0.692244       494



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.74603175, 0.10126582, 0.84140969, 0.4       , 0.9122807 ])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm.diagonal()

array([0.74603175, 0.10126582, 0.84140969, 0.4       , 0.9122807 ])

In [ ]:
#50 epochs teacher model

#alpha=0.9, temp=4

#50 epochs: 82.41
#60 epochs: 84.53
#70 epochs: 84.04

#alpha=0.9, temp=5

#50 epochs: 81.43
#60 epochs: 83.88
#70 epochs: 

#alpha=0.9, temp=3

#50 epochs: 83.88
#60 epochs: 83.06


KD using the 50 epochs teacher model

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 10s 7ms/step - loss: 1.7669 - accuracy: 0.7166


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.63492063, 0.07594937, 0.93832599, 0.4       , 0.92982456])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.6154757711561765
Recall: 0.5958041109204357
F1 score: 0.5797840921012746
              precision    recall  f1-score   support

           0   0.930233  0.634921  0.754717       126
           1   0.133333  0.075949  0.096774        79
           2   0.822394  0.938326  0.876543       227
           3   0.666667  0.400000  0.500000         5
           4   0.524752  0.929825  0.670886        57

    accuracy                       0.716599       494
   macro avg   0.615476  0.595804  0.579784       494
weighted avg   0.703786  0.716599  0.693229       494



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3710/3710 [==============================] - 25s 6ms/step - categorical_accuracy: 0.6954 - student_loss: 0.7428 - distillation_loss: 0.0016
Epoch 2/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7642 - student_loss: 0.5668 - distillation_loss: 0.0012
Epoch 3/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7756 - student_loss: 0.5426 - distillation_loss: 0.0012
Epoch 4/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7823 - student_loss: 0.5262 - distillation_loss: 0.0011
Epoch 5/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7910 - student_loss: 0.5064 - distillation_loss: 0.0011
Epoch 6/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0.7994 - student_loss: 0.4894 - distillation_loss: 0.0010
Epoch 7/60
3710/3710 [==============================] - 21s 6ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7470 - student_loss: 0.9388


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.77777778, 0.11392405, 0.92070485, 0.4       , 0.89473684])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.6110441188441189
Recall: 0.6214287032661862
F1 score: 0.5920252452648396
              precision    recall  f1-score   support

           0   0.907407  0.777778  0.837607       126
           1   0.321429  0.113924  0.168224        79
           2   0.836000  0.920705  0.876310       227
           3   0.500000  0.400000  0.444444         5
           4   0.490385  0.894737  0.633540        57

    accuracy                       0.746964       494
   macro avg   0.611044  0.621429  0.592025       494
weighted avg   0.728644  0.746964  0.720819       494



In [ ]:
#60 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 1.7056 - accuracy: 0.7267


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.63492063, 0.10126582, 0.95154185, 0.4       , 0.92982456])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.6302438357681023
Recall: 0.6035105738658435
F1 score: 0.5937700476745531
              precision    recall  f1-score   support

           0   0.930233  0.634921  0.754717       126
           1   0.160000  0.101266  0.124031        79
           2   0.824427  0.951542  0.883436       227
           3   0.666667  0.400000  0.500000         5
           4   0.569892  0.929825  0.706667        57

    accuracy                       0.726721       494
   macro avg   0.630244  0.603511  0.593770       494
weighted avg   0.714193  0.726721  0.704884       494



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3710/3710 [==============================] - 35s 8ms/step - categorical_accuracy: 0.6874 - student_loss: 0.7731 - distillation_loss: 0.0017
Epoch 2/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7673 - student_loss: 0.5571 - distillation_loss: 0.0012
Epoch 3/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7925 - student_loss: 0.5033 - distillation_loss: 0.0011
Epoch 4/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8194 - student_loss: 0.4470 - distillation_loss: 9.6651e-04
Epoch 5/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8324 - student_loss: 0.4185 - distillation_loss: 9.0098e-04
Epoch 6/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8378 - student_loss: 0.4043 - distillation_loss: 8.7086e-04
Epoch 7/70
3710/3710 [==============================] - 30s 8ms/step - categorical

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 5ms/step - categorical_accuracy: 0.7328 - student_loss: 1.2516


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.69047619, 0.06329114, 0.94713656, 0.4       , 0.92982456])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.5690676934765234
Recall: 0.6061456909993715
F1 score: 0.5640016562384983
              precision    recall  f1-score   support

           0   0.925532  0.690476  0.790909       126
           1   0.200000  0.063291  0.096154        79
           2   0.805243  0.947137  0.870445       227
           3   0.400000  0.400000  0.400000         5
           4   0.514563  0.929825  0.662500        57

    accuracy                       0.732794       494
   macro avg   0.569068  0.606146  0.564002       494
weighted avg   0.701493  0.732794  0.697580       494



In [ ]:
#40 epochs teacher model
#alpha=0.9, temp=4
#40 epochs: 83.71
#50 epochs: 84.36
#60 epochs: 83.88

In [ ]:
#70 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #70 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 2s 4ms/step - loss: 1.7825 - accuracy: 0.7227


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.61904762, 0.12658228, 0.96035242, 0.2       , 0.87719298])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.5980775145150559
Recall: 0.5566350605784522
F1 score: 0.5496683864050396
              precision    recall  f1-score   support

           0   0.917647  0.619048  0.739336       126
           1   0.185185  0.126582  0.150376        79
           2   0.825758  0.960352  0.887984       227
           3   0.500000  0.200000  0.285714         5
           4   0.561798  0.877193  0.684932        57

    accuracy                       0.722672       494
   macro avg   0.598078  0.556635  0.549668       494
weighted avg   0.713001  0.722672  0.702587       494



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=80)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/80
3710/3710 [==============================] - 32s 8ms/step - categorical_accuracy: 0.7120 - student_loss: 0.7217 - distillation_loss: 0.0015
Epoch 2/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7930 - student_loss: 0.5132 - distillation_loss: 0.0011
Epoch 3/80
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8100 - student_loss: 0.4716 - distillation_loss: 0.0010
Epoch 4/80
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8226 - student_loss: 0.4439 - distillation_loss: 9.6560e-04
Epoch 5/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8311 - student_loss: 0.4231 - distillation_loss: 9.2331e-04
Epoch 6/80
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8381 - student_loss: 0.4091 - distillation_loss: 8.8977e-04
Epoch 7/80
3710/3710 [==============================] - 29s 8ms/step - categorical

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.6680 - student_loss: 1.3287


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.42857143, 0.12658228, 0.92951542, 0.6       , 0.9122807 ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.5723511068370881
Recall: 0.599389965461806
F1 score: 0.5472500174705083
              precision    recall  f1-score   support

           0   0.981818  0.428571  0.596685       126
           1   0.153846  0.126582  0.138889        79
           2   0.811538  0.929515  0.866530       227
           3   0.428571  0.600000  0.500000         5
           4   0.485981  0.912281  0.634146        57

    accuracy                       0.668016       494
   macro avg   0.572351  0.599390  0.547250       494
weighted avg   0.708352  0.668016  0.650816       494



Trying out diff teachers and different students for perspective(the teacher who's epochs had been tuned)

In [ ]:
final_teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
final_teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_teacher_model.fit(x_dt_rshp, y_dt_, epochs=50)

Epoch 1/50
3710/3710 [==============================] - 49s 9ms/step - loss: 0.6716 - accuracy: 0.7297
Epoch 2/50
3710/3710 [==============================] - 33s 9ms/step - loss: 0.3967 - accuracy: 0.8436
Epoch 3/50
3710/3710 [==============================] - 33s 9ms/step - loss: 0.3286 - accuracy: 0.8693
Epoch 4/50
3710/3710 [==============================] - 32s 9ms/step - loss: 0.2876 - accuracy: 0.8861
Epoch 5/50
3710/3710 [==============================] - 32s 9ms/step - loss: 0.2541 - accuracy: 0.9006
Epoch 6/50
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2309 - accuracy: 0.9084
Epoch 7/50
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2138 - accuracy: 0.9164
Epoch 8/50
3710/3710 [==============================] - 33s 9ms/step - loss: 0.1980 - accuracy: 0.9214
Epoch 9/50
3710/3710 [==============================] - 32s 9ms/step - loss: 0.1898 - accuracy: 0.9250
Epoch 10/50
3710/3710 [==============================] - 32s 9ms/step - l

In [ ]:
accuracy_final_teacher_model = final_teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 9ms/step - loss: 1.7196 - accuracy: 0.7085


In [ ]:
preds_final_teacher_model = final_teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_teacher_model_fin=np.argmax(preds_final_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_teacher = confusion_matrix(y_test_dt, preds_final_teacher_model_fin)
matrix_final_teacher.diagonal()/matrix_final_teacher.sum(axis=1)

array([0.56349206, 0.15189873, 0.93832599, 0.4       , 0.9122807 ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_teacher = f1_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
prec_final_teacher = precision_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
recall_final_teacher = recall_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_teacher)) 
    print("Recall: {}".format(recall_final_teacher))
    print("F1 score: {}".format(f1_final_teacher))

    report_final_teacher = classification_report(y_test_dt, preds_final_teacher_model_fin, digits=6)
    print(report_final_teacher)

Precision: 0.6349819004524886
Recall: 0.5931994981226184
F1 score: 0.5873380635671208
              precision    recall  f1-score   support

           0   0.946667  0.563492  0.706468       126
           1   0.184615  0.151899  0.166667        79
           2   0.835294  0.938326  0.883817       227
           3   0.666667  0.400000  0.500000         5
           4   0.541667  0.912281  0.679739        57

    accuracy                       0.708502       494
   macro avg   0.634982  0.593199  0.587338       494
weighted avg   0.724058  0.708502  0.696464       494



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
final_student_model=get_model_1d_cnn()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 1497, 4)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=final_student_model, teacher=final_teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.7,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3710/3710 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6731 - student_loss: 0.8166 - distillation_loss: 0.0017
Epoch 2/60
3710/3710 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7632 - student_loss: 0.5758 - distillation_loss: 0.0012
Epoch 3/60
3710/3710 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7913 - student_loss: 0.5086 - distillation_loss: 0.0011
Epoch 4/60
3710/3710 [==============================] - 31s 8ms/step - categorical_accuracy: 0.8142 - student_loss: 0.4610 - distillation_loss: 9.6888e-04
Epoch 5/60
3710/3710 [==============================] - 31s 8ms/step - categorical_accuracy: 0.8267 - student_loss: 0.4331 - distillation_loss: 9.0993e-04
Epoch 6/60
3710/3710 [==============================] - 31s 8ms/step - categorical_accuracy: 0.8345 - student_loss: 0.4141 - distillation_loss: 8.6955e-04
Epoch 7/60
3710/3710 [==============================] - 31s 8ms/step - categorical

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 5ms/step - categorical_accuracy: 0.7409 - student_loss: 0.9654


In [ ]:
preds_final_student_model = final_student_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_student_model_fin=np.argmax(preds_final_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_student = confusion_matrix(y_test_dt, preds_final_student_model_fin)
matrix_final_student.diagonal()/matrix_final_student.sum(axis=1)

array([0.79365079, 0.11392405, 0.91629956, 0.4       , 0.8245614 ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_student = f1_score(y_test_dt, preds_final_student_model_fin, average="macro")
prec_final_student = precision_score(y_test_dt, preds_final_student_model_fin, average="macro")
recall_final_student = recall_score(y_test_dt, preds_final_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_student)) 
    print("Recall: {}".format(recall_final_student))
    print("F1 score: {}".format(f1_final_student))

    report_final_student = classification_report(y_test_dt, preds_final_student_model_fin, digits=6)
    print(report_final_student)

Precision: 0.5590325588066551
Recall: 0.6096871614527685
F1 score: 0.5664595532145201
              precision    recall  f1-score   support

           0   0.892857  0.793651  0.840336       126
           1   0.281250  0.113924  0.162162        79
           2   0.835341  0.916300  0.873950       227
           3   0.285714  0.400000  0.333333         5
           4   0.500000  0.824561  0.622517        57

    accuracy                       0.740891       494
   macro avg   0.559033  0.609687  0.566460       494
weighted avg   0.717145  0.740891  0.717064       494

